# Setup

Initial module setup.

In [1]:
import numpy.typing as np_types 

from sklearn.svm import LinearSVC
from eeg_auth_models_framework import data, pre_process, features, training, model

# Constants

In [2]:
DATASET_SAMPLE_FREQ_HZ = 200
DATA_CHANNEL_NAMES = ['T7','F8','Cz','P4']
FREQUENCIES = [
    pre_process.FrequencyBand(lower=8.0, upper=12.0, label='Alpha'),
    pre_process.FrequencyBand(lower=12.0, upper=35.0, label='Beta'),
    pre_process.FrequencyBand(lower=4.0, upper=8.0, label='Theta'),
    pre_process.FrequencyBand(lower=35.0, upper=None, label='Gamma'),
    pre_process.FrequencyBand(lower=None, upper=None, label='Raw'),
]
WINDOW_SIZE = 1200
WINDOW_OVERLAP = 0.5
K_FOLDS = 10

# Model Builder Configuration

Configure data source, data reading method, data labelling method, and training process. 

In [3]:
class ARSVMBuilder(model.ModelBuilder[LinearSVC]):
    @property
    def data_downloader(self):
        return data.AuditoryDataDownloader()
    
    @property
    def data_reader(self):
        return data.AuditoryDataReader()
    
    @property
    def labeller(self):
        return training.SubjectDataLabeller()
    
    def create_classifier(self) -> LinearSVC:
        return LinearSVC(
            random_state=32,
            dual='auto',
            max_iter=2000
        )
    
    def train_classifier(self, classifier: LinearSVC, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike):
        classifier.fit(x_data, y_data)
        
    def score_classifier(self, classifier: LinearSVC, x_data: np_types.ArrayLike, y_data: np_types.ArrayLike) -> float:
        return classifier.score(x_data, y_data)

# Pre-Processing Steps

Define pre-processing steps to be used in model.

In [4]:
pre_process_steps = pre_process.PreProcessingPipeline([
    pre_process.EEGBandpassFilterStep(
        FREQUENCIES, 
        DATA_CHANNEL_NAMES, 
        DATASET_SAMPLE_FREQ_HZ
    ),
    pre_process.DataWindowStep(WINDOW_SIZE, WINDOW_OVERLAP)
])

# Feature Extraction Steps

Define feature extraction steps to be applied to the pre-processed data.

In [5]:
feature_extraction_steps = features.FeatureExtractPipeline([
    features.ARFeatureExtractor({'lags': 25})
])

# Training

Execute training of authentication models.

In [6]:
ar_svm_builder = ARSVMBuilder(
    pre_process_steps,
    feature_extraction_steps
)
results = ar_svm_builder.train(K_FOLDS)
print('Average scores:')
for subject in results.models:
    print(f'{subject}: {results.get_average_score(subject)}')

Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=40114
    Range : 0 ... 40113 =      0.000 ...   200.565 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ...   119.995 secs
Ready.
Creating RawArray with float64 data, n_channels=4, n_times=24000
    Range : 0 ... 23999 =      0.000 ..